![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.4.Resume_MedicalNer_Model_Training.ipynb)

# 1.4 Resume MedicalNer Model Training

Steps:
- Train a new model for a few epochs.
- Load the same model and train for more epochs on the same taxnonomy, and check stats.
- Train a model already trained on a different data

## Colab Setup

In [ ]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4.1  spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [ ]:
# if you want to start the session with custom params as in start function above
from pyspark.sql import SparkSession

def start(SECRET):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:"+PUBLIC_VERSION) \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+SECRET+"/spark-nlp-jsl-"+JSL_VERSION+".jar")

    return builder.getOrCreate()

#spark = start(SECRET)

In [4]:
import json
import os

from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession

import sparknlp_jsl
import sparknlp

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *


import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", # Amount of memory to use for the driver process, i.e. where SparkContext is initialized
          "spark.kryoserializer.buffer.max":"2000M", # Maximum allowable size of Kryo serialization buffer, in MiB unless otherwise specified.
          "spark.driver.maxResultSize":"2000M"} # Limit of total size of serialized results of all partitions for each Spark action (e.g. collect) in bytes.
                                                # Should be at least 1M, or 0 for unlimited.

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 5.1.0
Spark NLP_JSL Version : 5.1.0


## Download Clinical Word Embeddings for training

In [5]:
clinical_embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


## Download Data for Training (NCBI Disease Dataset)

In [6]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NCBI_disease_official_test.conll
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NCBI_disease_official_train_dev.conll

In [7]:
from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, 'NCBI_disease_official_train_dev.conll')

training_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Identification of...|[{document, 0, 89...|[{document, 0, 89...|[{token, 0, 13, I...|[{pos, 0, 13, NN,...|[{named_entity, 0...|
|The adenomatous p...|[{document, 0, 21...|[{document, 0, 21...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
|Complex formation...|[{document, 0, 63...|[{document, 0, 63...|[{token, 0, 6, Co...|[{pos, 0, 6, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [8]:
from sparknlp.training import CoNLL

test_data = CoNLL().readDataset(spark, 'NCBI_disease_official_test.conll')

test_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Clustering of mis...|[{document, 0, 10...|[{document, 0, 10...|[{token, 0, 9, Cl...|[{pos, 0, 9, NN, ...|[{named_entity, 0...|
|Ataxia - telangie...|[{document, 0, 13...|[{document, 0, 13...|[{token, 0, 5, At...|[{pos, 0, 5, NN, ...|[{named_entity, 0...|
|The risk of cance...|[{document, 0, 15...|[{document, 0, 15...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



## Split the test data into two parts:
- We Keep the first part separate and use it for training the model further, as it will be totally unseen data from the same taxonomy.

- The second part will be used to testing and evaluating

In [9]:
(test_data_1, test_data_2) = test_data.randomSplit([0.5, 0.5], seed = 100)

# save the test data as parquet for easy testing
clinical_embeddings.transform(test_data_1).write.parquet('test_1.parquet')

clinical_embeddings.transform(test_data_2).write.parquet('test_2.parquet')

## Train a new model, pause, and resume training on the same dataset.

### Create a graph

We will use `TFGraphBuilder` annotator which can be used to create graphs in the model training pipeline. `TFGraphBuilder` inspects the data and creates the proper graph if a suitable version of TensorFlow (<= 2.7 ) is available. The graph is stored in the defined folder and loaded by the approach.

In [ ]:
!pip install -q tensorflow==2.12.0
!pip install -q tensorflow-addons

In [11]:
from sparknlp_jsl.annotator import TFGraphBuilder

In [12]:
graph_folder_path = "medical_ner_graphs"

ner_graph_builder = TFGraphBuilder()\
    .setModelName("ner_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder(graph_folder_path)\
    .setGraphFile("auto")\
    .setHiddenUnitsNumber(24)\
    .setIsLicensed(True) # False -> if you want to use TFGraphBuilder with NerDLApproach

In [ ]:
# You can create custom TensorFlow graph file (`.pb` extension) for NER model training externally by using the script below.
# If this method is used, graph folder should be added to MedicalNerApproach as  `.setGraphFolder(graph_folder_path)` .

'''
from sparknlp_jsl.training import tf_graph

tf_graph.print_model_params("ner_dl")

graph_folder_path = "medical_ner_graphs"
tf_graph.build("ner_dl",
               build_params={"embeddings_dim": 200,
                             "nchars": 128,
                             "ntags": 12,
                             "is_medical": 1},
                model_location=graph_folder_path,
                model_filename="auto")
'''

### Train for 2 epochs

In [14]:
nerTagger = MedicalNerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(2)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('./test_2.parquet')\
      .setGraphFolder(graph_folder_path)\
      .setOutputLogsPath('./ner_logs')

ner_pipeline = Pipeline(stages=[
      clinical_embeddings,
      ner_graph_builder,
      nerTagger
 ])

In [15]:
%%time
ner_model = ner_pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 24}


Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_85.pb
CPU times: user 18.4 s, sys: 749 ms, total: 19.2 s
Wall time: 2min 33s


In [16]:
!ls ner_logs/MedicalNerApproach*

ner_logs/MedicalNerApproach_eb4657c5dc32.log


In [17]:
# Training Logs
! cat ner_logs/MedicalNerApproach*

Name of the selected graph: /content/medical_ner_graphs/blstm_3_200_24_85.pb
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 6347


Epoch 1/2 started, lr: 0.003, dataset size: 6347


Epoch 1/2 - 49.36s - loss: 2072.3909 - avg training loss: 2.606781 - batches: 795
Quality on test dataset: 
time to finish evaluation: 2.06s
Total test loss: 88.0736	Avg test loss: 1.3762
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 475	 57	 155	 0.89285713	 0.75396824	 0.81755596
B-Disease	 402	 66	 129	 0.85897434	 0.75706214	 0.8048048
tp: 877 fp: 123 fn: 284 labels: 2
Macro-average	 prec: 0.87591577, rec: 0.7555152, f1: 0.8112726
Micro-average	 prec: 0.877, rec: 0.7553833, f1: 0.8116613


Epoch 2/2 started, lr: 0.0029850747, dataset size: 6347


Epoch 2/2 - 48.00s - loss: 956.3739 - avg training loss: 1.202986 - batches: 795
Quality on test dataset: 
time to finish evaluation: 1.12s
Total test loss: 77.5235	Avg test loss: 1.2113
label	 tp	 fp	 

In [18]:
# Logs of 4 consecutive epochs to compare with 2+2 epochs on separate datasets from same taxonomy

#!cat ner_logs/MedicalNerApproach_4d3d69967c3f.log

### Evaluate

In [19]:
from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

pred_df = ner_model.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+-----+-----+---------+------+------+
| entity|   tp|  fp|   fn|total|precision|recall|    f1|
+-------+-----+----+-----+-----+---------+------+------+
|Disease|417.0|73.0|110.0|527.0|    0.851|0.7913|0.8201|
+-------+-----+----+-----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8200589970501475|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8200589970501475|
+------------------+

None


`ner_utils`: This new module is used after NER training to calculate metric chunkbase and plot training logs.

`evaluate`: if verbose, returns overall performance, as well as performance per chunk type; otherwise, simply returns overall precision, recall, f1 scores

`loss_plot`: Plots the figure of loss vs epochs

`get_charts` : Plots the figures of metrics ( precision, recall, f1) vs epochs

```
from sparknlp_jsl.training_log_parser import ner_log_parser
parser = ner_log_parser()

pred_df = ner_model.stages[2].transform(test_data)
pred_df = pred_df.select(F.explode(F.arrays_zip(pred_df.ner.result,
                                                pred_df.label.result)).alias("cols"))\
                 .select(F.expr("cols['0']").alias("prediction"),
                         F.expr("cols['1']").alias("ground_truth"))
                 
df = pred_df.toPandas()

metrics = parser.evaluate( df['prediction'].values, df['ground_truth'].values)

parser.loss_plot(f"./ner_logs/{log_file}")

parser.get_charts('./ner_logs/'+log_file)
```

### Save the model to disk

In [20]:
ner_model.stages[2].write().overwrite().save('models/NCBI_NER_2_epoch')

### Train using the saved model on unseen dataset
#### We use unseen data from same taxonomy

In [21]:
nerTagger = MedicalNerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(2)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('/content/test_2.parquet')\
      .setOutputLogsPath('ner_logs')\
      .setGraphFolder(graph_folder_path)\
      .setPretrainedModelPath("models/NCBI_NER_2_epoch") ## load existing model

ner_pipeline = Pipeline(stages=[
      clinical_embeddings,
      ner_graph_builder,
      nerTagger
 ])

In [22]:
# remove the existing logs

! rm -r ./ner_logs

In [23]:
%%time
ner_model_retrained = ner_pipeline.fit(test_data_1)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 79, 'is_medical': True, 'lstm_size': 24}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_79.pb
CPU times: user 10.2 s, sys: 140 ms, total: 10.3 s
Wall time: 29.7 s


In [24]:
!ls ner_logs/MedicalNerApproach*

ner_logs/MedicalNerApproach_409a45bbff9c.log


In [25]:
!cat ./ner_logs/MedicalNerApproach*

Name of the selected graph: pretrained graph
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 438


Epoch 1/2 started, lr: 0.003, dataset size: 438


Epoch 1/2 - 6.33s - loss: 67.33217 - avg training loss: 1.1812661 - batches: 57
Quality on test dataset: 
time to finish evaluation: 1.93s
Total test loss: 57.1021	Avg test loss: 0.8922
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 518	 61	 112	 0.8946459	 0.82222223	 0.85690653
B-Disease	 456	 71	 75	 0.86527514	 0.8587571	 0.86200386
tp: 974 fp: 132 fn: 187 labels: 2
Macro-average	 prec: 0.87996054, rec: 0.8404896, f1: 0.85977226
Micro-average	 prec: 0.880651, rec: 0.838932, f1: 0.8592854


Epoch 2/2 started, lr: 0.0029850747, dataset size: 438


Epoch 2/2 - 3.97s - loss: 59.78158 - avg training loss: 1.0487996 - batches: 57
Quality on test dataset: 
time to finish evaluation: 1.22s
Total test loss: 58.7014	Avg test loss: 0.9172
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 519	 56	

In [26]:
from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

pred_df = ner_model_retrained.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|444.0|77.0|83.0|527.0|   0.8522|0.8425|0.8473|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8473282442748091|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8473282442748091|
+------------------+

None


### Train a Model Choosing Best Model

`useBestModel`:  This param preserves and restores the model that has achieved the best performance at the end of the training. The priority is metrics from testDataset (micro F1), metrics from validationSplit (micro F1), and if none is set it will keep track of loss during the training. <br/>

`earlyStopping`: You can stop training at the point when the perforfmance on test/validation dataset starts to degrage. Two params are used in order to use this feature: <br/>

- `earlyStoppingCriterion`: This is used set the minimal improvement of the test metric to terminate training. The metric monitored is the same as the metrics used in `useBestModel` (micro F1 when using test/validation set, loss otherwise). Default is 0 which means no early stopping is applied.

- `earlyStoppingPatience`:  The number of epoch without improvement which will be tolerated. Default is 0, which means that early stopping will occur at the first time when performance in the current epoch is no better than in the previous epoch.

In [27]:
# remove the existing logs

! rm -r ./ner_logs

In [28]:
nerTagger = MedicalNerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(20)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('./test_2.parquet')\
      .setGraphFolder(graph_folder_path)\
      .setOutputLogsPath('./ner_logs')\
      .setValidationSplit(0.2)\
      .setUseBestModel(True)\
      .setEarlyStoppingCriterion(0.04)\
      .setEarlyStoppingPatience(3)


ner_pipeline = Pipeline(stages=[
      clinical_embeddings,
      ner_graph_builder,
      nerTagger
 ])

In [29]:

%%time
ner_model = ner_pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 24}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_85.pb
CPU times: user 11.5 s, sys: 161 ms, total: 11.7 s
Wall time: 3min 23s


In [30]:
!ls ner_logs/MedicalNerApproach*

ner_logs/MedicalNerApproach_062f78463929.log


In [31]:
# Training Logs
! cat ner_logs/MedicalNerApproach*

Name of the selected graph: /content/medical_ner_graphs/blstm_3_200_24_85.pb
Training started - total epochs: 20 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 6347


Epoch 1/20 started, lr: 0.003, dataset size: 6347


Epoch 1/20 - 40.22s - loss: 2272.2441 - avg training loss: 3.572711 - batches: 636
Quality on validation dataset (20.0%), validation examples = 1269
time to finish evaluation: 3.72s
Total validation loss: 255.5190	Avg validation loss: 1.5676
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 1150	 207	 324	 0.84745765	 0.78018993	 0.8124337
B-Disease	 1098	 280	 152	 0.796807	 0.8784	 0.83561647
tp: 2248 fp: 487 fn: 476 labels: 2
Macro-average	 prec: 0.82213235, rec: 0.829295, f1: 0.82569814
Micro-average	 prec: 0.82193786, rec: 0.825257, f1: 0.8235941
Quality on test dataset: 
time to finish evaluation: 1.17s
Total test loss: 110.6474	Avg test loss: 1.7289
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 499	 97	 131	 0.8372483	 0.7920635	 0.8140294
B-Dise

As you see above, training was stopped before 20th epoch since there were not improvement.

**Evaluate**

In [32]:
from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

pred_df = ner_model.stages[2].transform(clinical_embeddings.transform(test_data_1))

evaler = NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|361.0|64.0|67.0|428.0|   0.8494|0.8435|0.8464|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8464243845252052|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8464243845252052|
+------------------+

None


In [33]:
pred_df = ner_model.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|433.0|88.0|94.0|527.0|   0.8311|0.8216|0.8263|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8263358778625954|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8263358778625954|
+------------------+

None


## Now let's take a model trained on a different dataset and train on this dataset

In [34]:
jsl_ner = MedicalNerModel.pretrained('ner_jsl','en','clinical/models')

jsl_ner.getClasses()

ner_jsl download started this may take some time.
[OK!]


['O',
 'B-Injury_or_Poisoning',
 'B-Direction',
 'B-Test',
 'I-Route',
 'B-Admission_Discharge',
 'B-Death_Entity',
 'I-Oxygen_Therapy',
 'B-Relationship_Status',
 'I-Drug_BrandName',
 'B-Duration',
 'I-Alcohol',
 'I-Triglycerides',
 'I-Date',
 'B-Hyperlipidemia',
 'B-Respiration',
 'I-Test',
 'B-Birth_Entity',
 'I-VS_Finding',
 'B-Age',
 'I-Vaccine_Name',
 'I-Social_History_Header',
 'B-Labour_Delivery',
 'I-Medical_Device',
 'B-Family_History_Header',
 'B-BMI',
 'I-Fetus_NewBorn',
 'I-BMI',
 'B-Temperature',
 'I-Section_Header',
 'I-Communicable_Disease',
 'I-ImagingFindings',
 'I-Psychological_Condition',
 'I-Obesity',
 'I-Sexually_Active_or_Sexual_Orientation',
 'I-Modifier',
 'B-Alcohol',
 'I-Temperature',
 'I-Vaccine',
 'I-Symptom',
 'I-Pulse',
 'B-Kidney_Disease',
 'B-Oncological',
 'I-EKG_Findings',
 'B-Medical_History_Header',
 'I-Relationship_Status',
 'B-Cerebrovascular_Disease',
 'I-Blood_Pressure',
 'I-Diabetes',
 'B-Oxygen_Therapy',
 'B-O2_Saturation',
 'B-Psychological_C

### Now train a model using this model as base

In [35]:
nerTagger = MedicalNerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(2)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('/content/test_2.parquet')\
      .setOutputLogsPath('ner_logs')\
      .setGraphFolder(graph_folder_path)\
      .setPretrainedModelPath("/root/cache_pretrained/ner_jsl_en_4.2.0_3.0_1666181370373")\
      .setOverrideExistingTags(True) # since the tags do not align, set this flag to true

# do hyperparameter by tuning the params above (max epoch, LR, dropout etc.) to get better results
ner_pipeline = Pipeline(stages=[
      clinical_embeddings,
      ner_graph_builder,
      nerTagger
 ])

In [36]:
# remove the existing logs

! rm -r ./ner_logs

In [37]:

%%time
ner_jsl_retrained = ner_pipeline.fit(training_data)


TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 24}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_85.pb
CPU times: user 13 s, sys: 286 ms, total: 13.3 s
Wall time: 6min 15s


In [38]:
!cat ./ner_logs/MedicalNerApproach*

Name of the selected graph: pretrained graph
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 103 - training examples: 6347


Epoch 1/2 started, lr: 0.003, dataset size: 6347


Epoch 1/2 - 163.47s - loss: 1364.9674 - avg training loss: 1.7169402 - batches: 795
Quality on test dataset: 
time to finish evaluation: 13.07s
Total test loss: 66.5383	Avg test loss: 1.0397
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 519	 57	 111	 0.9010417	 0.8238095	 0.8606965
B-Disease	 466	 83	 65	 0.84881604	 0.87758946	 0.862963
tp: 985 fp: 140 fn: 176 labels: 2
Macro-average	 prec: 0.87492883, rec: 0.8506995, f1: 0.862644
Micro-average	 prec: 0.8755556, rec: 0.84840655, f1: 0.8617673


Epoch 2/2 started, lr: 0.0029850747, dataset size: 6347


Epoch 2/2 - 156.58s - loss: 643.5219 - avg training loss: 0.80946153 - batches: 795
Quality on test dataset: 
time to finish evaluation: 11.58s
Total test loss: 63.8708	Avg test loss: 0.9980
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disea

In [39]:
pred_df = ner_jsl_retrained.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+-----+-----+---------+------+------+
| entity|   tp|  fp|   fn|total|precision|recall|    f1|
+-------+-----+----+-----+-----+---------+------+------+
|Disease|416.0|64.0|111.0|527.0|   0.8667|0.7894|0.8262|
+-------+-----+----+-----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8262164846077458|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8262164846077458|
+------------------+

None
